In [1]:
import os
os.getcwd()
os.chdir('D:\\Git projects\\college_works\\Thesis')

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from math import gamma
import time
import pandas as pd

In [2]:
def par():
    global beta, eta, varphi, theta, rho, i, r, gamma1, phi, kappa, alfa, psi, sig, pi, nu
    beta = 0.69
    eta = 0.25
    varphi = 0.25
    theta = 3.44
    rho = 0.19
    alfa = 0.6
    kappa = np.divide(1, (1- eta) )
    nu = 1 + np.multiply(alfa, np.multiply(varphi, kappa)) - np.divide(kappa, theta)
    pi = 1 - np.multiply(np.multiply( (1 - alfa), varphi), kappa )    
    sig = np.divide(np.multiply(eta, kappa), pi )
    psi = np.multiply(np.power(eta, eta), np.power( (1-eta), (1-eta) ) )
    i = 7
    r = 27
    gamma1 = gamma(   1 - np.multiply( np.divide(1, np.multiply(theta, (1-rho)) ), np.divide(1, (1 - eta) ) ) )   
    phi = np.array([0.138, 0.174, 0.136, 0.1, 0.051, 0.084, 0.168]).reshape(i, 1)

In [3]:

def taus2():
    par()
        
    tau_h = np.random.uniform(low=-0.99, high=40, size=(i,r))
    tau_h[0, :] = 0

    tau_w = np.random.uniform(low=-0.99, high=0.999, size=(i,r))
    tau_w[0, : ] = tau_w[0, 0]
    
    w =np.random.uniform(low=0.001, high=30, size=(i,r))
    w[0, r-1] = 1
    #w[1:6, :] = w[0, :] 
    
    x1 = np.array([tau_w, tau_h, w])
    
    return x1

x1 = taus2()

In [4]:
def sf( ):
    global s
    s = np.power( (1+ np.divide( (1-eta), ( np.multiply(beta, phi) ) ) ), -1 )
    s = s.reshape(i, 1)
    return s

In [5]:
def p_trf(x1):    
    s = sf( )    
    A = np.divide( (1 - x1[0]) , np.power( ( 1 + x1[1] ), eta) )     
    b = np.power(s, phi ) 
    B = np.multiply(b.reshape(i,1), x1[2] )     
    C = np.power( (1 - s), np.divide((1-eta), beta) )
    d = np.multiply(np.multiply(A, B), C )
    k = np.power(d, theta)
    p_tr = np.divide( k[i-1], np.sum(k, axis=0) )
    return p_tr
 

In [6]:
  
def H_trf(x1):    
    p_tr = p_trf(x1)  
    A = np.power(np.multiply( np.divide( (1 - x1[0]), (1 + x1[1]) ), x1[2] ), sig )    
    A = A[i-1]     
    P = np.multiply(np.power(p_tr, ( np.divide(nu, pi) )), np.power(eta, sig ) )
    g = np.divide(np.multiply(phi[i-1], kappa ), pi )
    C = np.multiply( np.power(s[i-1], g ), np.power(gamma1, np.divide(1, pi) ) )
    H_tr = np.multiply(np.multiply(P, A), C )       
    return H_tr

In [7]:
def w_tilf(x1):
    H_tr = H_trf(x1) 
    p_tr = p_trf(x1)                 
    C = np.power((1 - s), np.divide( (1-eta), beta )).reshape(7, 1)
    pp = np.power(s, phi)        
    A = np.multiply(psi, np.multiply( np.divide( (1 - x1[0]), np.power( (1 + x1[1]), eta ) ), x1[2] ) )    
    b = np.power(np.multiply(np.power(p_tr, alfa), np.power(H_tr, (1-alfa) ) ), varphi )    
    B = np.multiply(b, pp.reshape(7,1))
    w_til =  np.multiply(np.multiply(A, B), C )
    return w_til 

In [8]:

def p_irf(x1):    
    w_til = w_tilf(x1)
    w_til2 = np.power(w_til, theta) 
    w_r = w_til2.sum(axis = 0)    
    p_ir = np.divide(w_til2 , w_r ) 
    return np.array(p_ir), np.array(w_r)

In [9]:
def Wf(x1):
    p_ir, w_r = p_irf(x1)         
    z = np.multiply(np.multiply(gamma1, eta), np.power(w_r, np.divide(1, np.multiply(theta, (1 - eta)) ) ) )
    A = np.divide( np.power( (1 - s), (-1/beta) ), ( 1 - x1[0] )  )                
    W = np.multiply(A, z)
    return W, p_ir

In [10]:

def simul():
    global p_t, W_t
    p_t = pd.read_csv('pt.csv', sep=';')
    p_t = p_t.iloc[0:7]
    p_t.set_index('ocup', inplace=True)
    p_t = np.array(p_t)
    W_t = pd.read_csv('wt.csv', sep=';')
    W_t.set_index('ocup', inplace=True)
    W_t = np.array(W_t)
    
    return p_t, W_t

simul();

In [11]:


p_t2 = np.array(p_t[0:6])
    
def obj2(x1):       
    x1 = x1.reshape((3, i, r)) 
    x1[0, 0, : ] = x1[0, 0, 0]    
    x1[1, 0, :] = 0
    x1[2, :, 16] = 8
    x1[2, 0:7, :] = x1[2, 0, :]
    W, p_ir = Wf(x1)    
    p_ir = p_ir[0:6]
    D =   (np.power(np.divide( (W-W_t), W_t ), 2)).sum() + (np.power(np.divide( (p_ir-p_t2), p_t2 ), 2)).sum() 
    D = np.log(D)    
    return D

In [12]:
#%time obj2(np.vstack(taus2()).flatten())
%time obj2(taus2())

In [13]:


# optimization

Bd = ((-0.99, 0.999), )*189 + ((-0.99, 40), )*189 + ((0.001, 12), )*27
Bd = np.array(Bd, dtype=object)

def hessp(x, l):
    return np.zeros((3, i, r))

In [14]:
# optimization give me z1 array

z1 = pd.read_excel('MG_8.xlsx')
z1 = np.array(z1).reshape(3, i, r)


obj2(z1)

In [15]:
def h_tilf(x1):
    H_tr = H_trf(x1) 
    p_tr = p_trf(x1) 
    A = np.power(np.multiply(np.power(p_tr, alfa), np.power(H_tr, (1-alfa) ) ), varphi )    
    B = np.multiply(np.power(s, phi), np.power(eta, eta) )
    h_til = np.power(np.multiply(A, B), kappa )
    return h_til

In [16]:
def H_irf(x1):
    p_ir = p_irf(x1)[0]
    h_til = h_tilf(x1)
    A = np.multiply(h_til, np.power(p_ir, (1-kappa/theta) ) )
    B = np.power(np.multiply( np.divide( (1 - x1[0]), (1 + x1[1]) ), x1[2] ), np.multiply(eta, kappa ) )       
    H_ir = np.multiply(np.multiply(A, B), gamma1 )
    return H_ir
    
    

In [17]:


def Y_f(x1):
    H_ir = H_irf(x1) 
    Y = np.multiply(x1[2, 0, :], H_ir)
    return Y


    

In [18]:

# Model

W, p_ir = Wf(z1.reshape(3, i, r))
W = pd.DataFrame(W)
p_ir = pd.DataFrame(p_ir)

 
# PNAD data

p_t, W_t = simul()   
W_t =  pd.DataFrame(W_t)
p_t =  pd.DataFrame(p_t)


# get colnames

n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names = n.columns.str.strip("'")
names  = np.array(names).repeat(7) 



p_t = p_t.values.flatten() 
p_ir = p_ir.values.flatten()

W = W.values.flatten()
W_t = W_t.values.flatten()

In [19]:
# plots

### W and W_ir

W_t =  pd.DataFrame(W_t)
W = pd.DataFrame(W)


W = W.values.flatten()
W_t = W_t.values.flatten()


ocn1 = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']
ocn1 = np.array(ocn1, dtype=object).repeat(27)

dd = pd.DataFrame(dict(x=W, y=W_t, label=ocn1))
groups = dd.groupby('label')


## p_ir and p_t

p_t =  pd.DataFrame(p_t)
p_ir = pd.DataFrame(p_ir)


p_t = p_t.values.flatten() 
p_ir = p_ir.values.flatten()

ocn = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']


ocn = np.array(ocn, dtype=object).repeat(27)


ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
groups2 = ff.groupby('label')

In [20]:

# gdp model

gdp_1 = Y_f(z1).sum(axis=0) 
names2 = np.array(names2)
data1 = {'state': names2.flatten(), 'GDP_model': gdp_1.flatten()}

gdp_model = pd.DataFrame(data1)

# gdp data

gdp_data = pd.read_excel('gdp_data.xlsx')

gdp_data['GDP_data'] = np.log(gdp_data['GDP_data'])


### merge data frames

GDP = pd.merge(gdp_model, gdp_data, on='state')
GDP.head()

GDP_model = GDP['GDP_model']
GDP_data = GDP['GDP_data']
GDP_names = GDP['state']

In [21]:
# plots

### W and W_ir

W_t =  pd.DataFrame(W_t)
W = pd.DataFrame(W)


W = W.values.flatten()
W_t = W_t.values.flatten()


ocn1 = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']
ocn1 = np.array(ocn1, dtype=object).repeat(27)

dd = pd.DataFrame(dict(x=W, y=W_t, label=ocn1))
groups = dd.groupby('label')


## p_ir and p_t

p_t =  pd.DataFrame(p_t)
p_ir = pd.DataFrame(p_ir)


p_t = p_t.values.flatten() 
p_ir = p_ir.values.flatten()

ocn = ['Managers','Sciences and arts','Middle-level technicians','Service-sector','Agriculture',
       'Industrial production and services','Teachers']


ocn = np.array(ocn, dtype=object).repeat(27)


ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
groups2 = ff.groupby('label')

In [22]:

# Model

W, p_ir = Wf(z1.reshape(3, i, r))
W = pd.DataFrame(W)
p_ir = pd.DataFrame(p_ir)

 
# PNAD data

p_t, W_t = simul()   
W_t =  pd.DataFrame(W_t)
p_t =  pd.DataFrame(p_t)


# get colnames

n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names = n.columns.str.strip("'")
names  = np.array(names).repeat(7) 



p_t = p_t.values.flatten() 
p_ir = p_ir.values.flatten()

W = W.values.flatten()
W_t = W_t.values.flatten()

In [23]:

# gdp model

gdp_1 = Y_f(z1).sum(axis=0) 
names2 = np.array(names2)
data1 = {'state': names2.flatten(), 'GDP_model': gdp_1.flatten()}

gdp_model = pd.DataFrame(data1)

# gdp data

gdp_data = pd.read_excel('gdp_data.xlsx')

gdp_data['GDP_data'] = np.log(gdp_data['GDP_data'])


### merge data frames

GDP = pd.merge(gdp_model, gdp_data, on='state')
GDP.head()

GDP_model = GDP['GDP_model']
GDP_data = GDP['GDP_data']
GDP_names = GDP['state']

In [24]:

# gdp model

gdp_1 = Y_f(z1).sum(axis=0) 
names2 = np.array(names)
data1 = {'state': names2.flatten(), 'GDP_model': gdp_1.flatten()}

gdp_model = pd.DataFrame(data1)

# gdp data

gdp_data = pd.read_excel('gdp_data.xlsx')

gdp_data['GDP_data'] = np.log(gdp_data['GDP_data'])


### merge data frames

GDP = pd.merge(gdp_model, gdp_data, on='state')
GDP.head()

GDP_model = GDP['GDP_model']
GDP_data = GDP['GDP_data']
GDP_names = GDP['state']

In [25]:
names

In [26]:
names[0]

In [27]:
names[0:27]

In [28]:
names[1,:]

In [29]:
names[1:]

In [30]:
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")
#names  = np.array(names).repeat(7) 

In [31]:
n = pd.read_csv('pt.csv', sep=';')
n = n.iloc[0:7]
n.set_index('ocup', inplace=True)
names2 = n.columns.str.strip("'")
#names  = np.array(names).repeat(7) 

names2

In [32]:

# gdp model

gdp_1 = Y_f(z1).sum(axis=0) 
names2 = np.array(names2)
data1 = {'state': names2.flatten(), 'GDP_model': gdp_1.flatten()}

gdp_model = pd.DataFrame(data1)

# gdp data

gdp_data = pd.read_excel('gdp_data.xlsx')

gdp_data['GDP_data'] = np.log(gdp_data['GDP_data'])


### merge data frames

GDP = pd.merge(gdp_model, gdp_data, on='state')
GDP.head()

GDP_model = GDP['GDP_model']
GDP_data = GDP['GDP_data']
GDP_names = GDP['state']

In [33]:
GDP.head()

In [34]:
GDP.head(10)

In [35]:
GDP.head(10).sort()

In [36]:
GDP.head(10)

In [37]:

# gpd per worker and hc - fig5

H_tr = H_trf(z1)
Y = Y_f(z1).sum(axis=0)
names2 = n.columns.str.strip("'")

plt.subplots(figsize=(10,8))
plt.scatter(H_tr, Y, s=10)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
(m, b) = np.polyfit(H_tr, Y, 1)
yp = np.polyval([m, b], H_tr)
plt.plot(H_tr, yp, label='Regression Line')
for tt, txt in enumerate(names2):
    plt.annotate(txt, (H_tr[tt], Y[tt]), size=20) 
plt.grid(True)
plt.legend(loc="upper right", prop={'size': 20})
plt.xlabel(" Human capital of teachers - Model", fontsize=20)
plt.ylabel("GDP per worker - Model", fontsize=20)
plt.tight_layout()    
plt.savefig("D:/DOUTORADO/Topics in macro/paper_tex/fig4.eps", format='eps', dpi=1200)

In [38]:
GDP.head(10)

GDP2 = GDP.copy

In [39]:
GDP2.head(10)

In [40]:
GDP2.head(10)

In [41]:
GDP2

In [42]:
GDP2

GDP2 = GDP.copy()

In [43]:
GDP2

#GDP2 = GDP.copy()

In [44]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['code']

In [45]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['code']==1

In [46]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2.loc[[GDP2['code']==1], ['code'] ]

In [47]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2.loc[ [GDP2['code']==1], : ]

In [48]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2.iloc[ [GDP2['code']==1], ['code'] ]

In [49]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2.loc[ GDP2['code']==1, ['code'] ]

In [50]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0

#GDP2.loc[ GDP2['code']==1, ['code'] ]

In [51]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0

#GDP2.loc[ GDP2['code']==1, ['code'] ]

GDP2

In [52]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0

GDP2.loc[ GDP2['code']==1, ['code'] ]

In [53]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0
bol1 = GDP2['code']==1

GDP2.loc[bol1 , ['income'] ]

In [54]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0

bol1 = GDP2['code']==1
GDP2.loc[bol1 , ['income'] ] = 'High Income'


bol2 = GDP2['code'] == 2
GDP2.loc[bol2 , ['income'] ] = 'Middle Income'


bol3 = GDP2['code'] == 3
GDP2.loc[bol3 , ['income'] ] = 'Low Income'

In [55]:
GDP2

In [56]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0

bol1 = GDP2['code']==1
GDP2.loc[bol1 , ['income'] ] = 'High Income'


bol2 = GDP2['code'] == 2
GDP2.loc[bol2 , ['income'] ] = 'Middle Income'


bol3 = GDP2['code'] == 3
GDP2.loc[bol3 , ['income'] ] = 'Low Income'

group3 = GDP2.groupby('income')

In [57]:
group3

In [58]:

#ff = pd.DataFrame(dict(x=p_ir, y=p_t, label=ocn))
#groups2 = ff.groupby('label')

GDP2['income'] = 0

bol1 = GDP2['code']==1
GDP2.loc[bol1 , ['income'] ] = 'High Income'


bol2 = GDP2['code'] == 2
GDP2.loc[bol2 , ['income'] ] = 'Middle Income'


bol3 = GDP2['code'] == 3
GDP2.loc[bol3 , ['income'] ] = 'Low Income'

group3 = GDP2.groupby('income')

GDP2.head()

In [59]:
ax, fig = plt.subplots(figsize=(10,8))
ax[1].margins(0.05) 
for ocn, group in groups3:
    ax[1].plot(grou3.GDP_model, group3.GDP_data, marker='o', linestyle='', ms=12, label=ocn)
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
for ii, txt in enumerate(names2):
    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [60]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for ocn, group in groups3:
    ax[1].plot(grou3.GDP_model, group3.GDP_data, marker='o', linestyle='', ms=12, label=ocn)
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
for ii, txt in enumerate(names2):
    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [61]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for ocn, group in group3:
    ax[1].plot(group3.GDP_model, group3.GDP_data, marker='o', linestyle='', ms=12, label=ocn)
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
for ii, txt in enumerate(names2):
    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [62]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax[1].plot(group3.GDP_model, group3.GDP_data, marker='o', linestyle='', ms=12, label=income)
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
for ii, txt in enumerate(names2):
    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [63]:
ax, fig = plt.subplots(1, 2, figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax[1].plot(group3.GDP_model, group3.GDP_data, marker='o', linestyle='', ms=12, label=income)
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
for ii, txt in enumerate(names2):
    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [64]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax[1].plot(group3.GDP_model, group3.GDP_data, marker='o', linestyle='', ms=12, label=income)
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [65]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax[1].plot(group.GDP_model, group.GDP_data, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [66]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.GDP_model, group.GDP_data, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [67]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

In [68]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

groups3 

In [69]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

In [70]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.GDP_model, group.GDP_data, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [71]:
ax, fig = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [72]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [73]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [74]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

group3()

In [75]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

group3

In [76]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

group3.head

In [77]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

group3.head()

In [78]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('income')

group3.head()

In [79]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

group3.head()

In [80]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

group3.head()

In [81]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

#group3.head()
zz

In [82]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')
zz.groupby('label')

In [83]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
groups3 = zz.groupby('label')

In [84]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [85]:


zz = pd.DataFrame(dict(x=GDP2['GDP_model'], y=GDP2['GDP_data'], label=GDP2['income']))
group3 = zz.groupby('label')

In [86]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)

    
ax[1].xaxis.set_tick_params(labelsize=20)
ax[1].yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [87]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)

    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [88]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
for ii, txt in enumerate(names2):
    ax.annotate(txt, (group.x[ii], group.y[ii]), size=20) 

ax.legend(loc="upper left", prop={'size': 12})






#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [89]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
#for ii, txt in enumerate(names2):
#    ax.annotate(txt, (group.x[ii], group.y[ii]), size=20) 

ax.legend(loc="upper left", prop={'size': 12})






#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [90]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper left", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (group.x[ii], group.y[ii]), size=20) 







#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [91]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper left", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (GDP2['GDP_model'][ii], GDP2['GDP_data'][ii]), size=20) 







#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [92]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group3:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper left", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (GDP2['GDP_model'][ii], GDP2['GDP_data'][ii]), size=20) 
ax.grid(True)






#for ii, txt in enumerate(names2):
#    ax[1].annotate(txt, (p_ir[ii], p_t[ii]), size=20) 

In [93]:

# gpd per worker and hc 

H_tr = H_trf(z1)
Y = Y_f(z1).sum(axis=0)
names2 = n.columns.str.strip("'")

data1 = {'H_tr': H_tr, 'Y': Y, 'state': names2}

HC = pd.merge(data1, GDP2, on='state')

In [94]:

# gpd per worker and hc 

H_tr = H_trf(z1)
Y = Y_f(z1).sum(axis=0)
names2 = n.columns.str.strip("'")

data1 = {'H_tr': H_tr, 'Y': Y, 'state': names2}
data1 = pd.DataFrame(data1)

HC = pd.merge(data1, GDP2, on='state')

In [95]:

# gpd per worker and hc 

H_tr = H_trf(z1)
Y = Y_f(z1).sum(axis=0)
names2 = n.columns.str.strip("'")

data1 = {'H_tr': H_tr, 'Y': Y, 'state': names2}
data1 = pd.DataFrame(data1)

HC = pd.merge(data1, GDP2, on='state')

HC

In [96]:

# gpd per worker and hc 

H_tr = H_trf(z1)
Y = Y_f(z1).sum(axis=0)
names2 = n.columns.str.strip("'")

data1 = {'H_tr': H_tr, 'Y': Y, 'state': names2}
data1 = pd.DataFrame(data1)

HC = pd.merge(data1, GDP2, on='state')



hh = pd.DataFrame(dict(x=HC['H_tr'], y=HC['GDP_data'], label=GDP2['income']))
group4 = hh.groupby('label')

In [97]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper left", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], GDP2['GDP_data'][ii]), size=20) 
ax.grid(True)

In [98]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)    
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], GDP2['GDP_data'][ii]), size=20) 
ax.grid(True)

In [99]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
yp = np.polyval([m, b], H_tr)
ax.plot(HC['H_tr'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], GDP2['GDP_data'][ii]), size=20) 
ax.grid(True)

In [100]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
yp = np.polyval([m, b], H_tr)
ax.plot(HC['H_tr'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], GDP2['GDP_data'][ii]), size=20) 
ax.grid(True)
plt.tight_layout()    

In [101]:

# gpd per worker and hc 

H_tr = H_trf(z1)
Y = Y_f(z1).sum(axis=0)
names2 = n.columns.str.strip("'")

data1 = {'H_tr': H_tr, 'Y': Y, 'state': names2}
data1 = pd.DataFrame(data1)

HC = pd.merge(data1, GDP2, on='state')



hh = pd.DataFrame(dict(x=HC['H_tr'], y=HC['GDP_model'], label=GDP2['income']))
group4 = hh.groupby('label')

In [102]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
yp = np.polyval([m, b], HC['H_tr'])
ax.plot(HC['H_tr'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})

for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], GDP2['GDP_model'][ii]), size=20) 
ax.grid(True)
plt.tight_layout()    

In [103]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
yp = np.polyval([m, b], HC['H_tr'])
ax.plot(HC['H_tr'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], HC['GDP_model'][ii]), size=20) 
ax.set_xlabel("GDP - Model", fontsize=20)
ax.set_ylabel("TFP - Model", fontsize=20)
ax.grid(True)
plt.tight_layout()    

In [104]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
yp = np.polyval([m, b], HC['H_tr'])
ax.plot(HC['H_tr'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], HC['GDP_model'][ii]), size=20) 
ax.set_xlabel("Human Capital - Model", fontsize=20)
ax.set_ylabel("GDP - Model", fontsize=20)
ax.grid(True)
plt.tight_layout()    

In [105]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
yp = np.polyval([m, b], HC['H_tr'])
ax.plot(HC['H_tr'], yp, color='black', label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], HC['GDP_model'][ii]), size=20) 
ax.set_xlabel("Human Capital - Model", fontsize=20)
ax.set_ylabel("GDP - Model", fontsize=20)
ax.grid(True)
plt.tight_layout()    

In [106]:

## p_ir x teachers wage - fig4


p_tr = p_trf(z1)

twg = np.array(W).reshape(i, r)[(i-1), :]/np.array(W).reshape(i, r).sum(axis=0)
 


names2 = n.columns.str.strip("'")

plt.subplots(figsize=(10,8))
plt.scatter(twg, p_tr, s=10)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
(m, b) = np.polyfit(twg, p_tr, 1)
yp = np.polyval([m, b], twg)
plt.plot(twg, yp, label='Regression Line')
for tt, txt in enumerate(names2):
    plt.annotate(txt, (twg[tt], p_tr[tt]), size=20) 
plt.grid(True)
plt.legend(loc="upper left", prop={'size': 20})
plt.xlabel(" Teachers wage relative to other occupations - Model", fontsize=20)
plt.ylabel("Proportion of teachers - Model", fontsize=20)
plt.tight_layout()    
plt.savefig("C:/Users/user/Downloads/paper_missalocation/paper/fig5.eps", format='eps', dpi=1200)


  


 

In [107]:

## p_ir x teachers wage - fig4


p_tr = p_trf(z1)

twg = np.array(W).reshape(i, r)[(i-1), :]/np.array(W).reshape(i, r).sum(axis=0)
 


names2 = n.columns.str.strip("'")

plt.subplots(figsize=(10,8))
plt.scatter(twg, p_tr, s=10)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
(m, b) = np.polyfit(twg, p_tr, 1)
yp = np.polyval([m, b], twg)
plt.plot(twg, yp, label='Regression Line')
for tt, txt in enumerate(names2):
    plt.annotate(txt, (twg[tt], p_tr[tt]), size=20) 
plt.grid(True)
plt.legend(loc="upper left", prop={'size': 20})
plt.xlabel(" Teachers wage relative to other occupations - Model", fontsize=20)
plt.ylabel("Proportion of teachers - Model", fontsize=20)
plt.tight_layout()    
#plt.savefig("C:/Users/user/Downloads/paper_missalocation/paper/fig5.eps", format='eps', dpi=1200)


  


 

In [108]:

# gpd per worker and hc 

## p_ir x teachers wage - fig4


p_tr = p_trf(z1)

twg = np.array(W).reshape(i, r)[(i-1), :]/np.array(W).reshape(i, r).sum(axis=0)
 
    
names2 = n.columns.str.strip("'")

data2 = {'p_tr': p_tr, 'twg': twg, 'state': names2}
data2 = pd.DataFrame(data2)

TW = pd.merge(data2, GDP2, on='state')



ww = pd.DataFrame(dict(x=TW['P_tr'], y=TW['twg'], label=TW['income']))
group5 = ww.groupby('label')

In [109]:

# gpd per worker and hc 

## p_ir x teachers wage - fig4


p_tr = p_trf(z1)

twg = np.array(W).reshape(i, r)[(i-1), :]/np.array(W).reshape(i, r).sum(axis=0)
 
    
names2 = n.columns.str.strip("'")

data2 = {'p_tr': p_tr, 'twg': twg, 'state': names2}
data2 = pd.DataFrame(data2)

TW = pd.merge(data2, GDP2, on='state')



ww = pd.DataFrame(dict(x=TW['p_tr'], y=TW['twg'], label=TW['income']))
group5 = ww.groupby('label')

In [110]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
#(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
#yp = np.polyval([m, b], HC['H_tr'])
#ax.plot(HC['H_tr'], yp, color='black', label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['p_tr'][ii], TW['twg'][ii]), size=20) 
ax.set_xlabel("Proportion of teachers - Model", fontsize=20)
ax.set_ylabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [111]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
#(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
#yp = np.polyval([m, b], HC['H_tr'])
#ax.plot(HC['H_tr'], yp, color='black', label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="lower right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['p_tr'][ii], TW['twg'][ii]), size=20) 
ax.set_xlabel("Proportion of teachers - Model", fontsize=20)
ax.set_ylabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [112]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(TW['twg'], TW['p_tr'], 1)
yp = np.polyval([m, b], TW['twg'])
ax.plot(TW['twg'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="lower right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['p_tr'][ii], TW['twg'][ii]), size=20) 
ax.set_xlabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.set_ylabel("Proportion of teachers - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [113]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(TW['twg'], TW['p_tr'], 1)
yp = np.polyval([m, b], TW['twg'])
ax.plot(TW['twg'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="lower right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['twg'][ii], TW['p_tr'][ii]), size=20) 
ax.set_xlabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.set_ylabel("Proportion of teachers - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [114]:

# gpd per worker and hc 

## p_ir x teachers wage - fig4


p_tr = p_trf(z1)

twg = np.array(W).reshape(i, r)[(i-1), :]/np.array(W).reshape(i, r).sum(axis=0)
 
    
names2 = n.columns.str.strip("'")

data2 = {'p_tr': p_tr, 'twg': twg, 'state': names2}
data2 = pd.DataFrame(data2)

TW = pd.merge(data2, GDP2, on='state')



ww = pd.DataFrame(dict(x=TW['twg'], y=TW['p_tr'], label=TW['income']))
group5 = ww.groupby('label')

In [115]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(TW['twg'], TW['p_tr'], 1)
yp = np.polyval([m, b], TW['twg'])
ax.plot(TW['twg'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="lower right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['twg'][ii], TW['p_tr'][ii]), size=20) 
ax.set_xlabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.set_ylabel("Proportion of teachers - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [116]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(TW['twg'], TW['p_tr'], 1)
yp = np.polyval([m, b], TW['twg'])
ax.plot(TW['twg'], yp, label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper left", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['twg'][ii], TW['p_tr'][ii]), size=20) 
ax.set_xlabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.set_ylabel("Proportion of teachers - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [117]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
yp = np.polyval([m, b], HC['H_tr'])
ax.plot(HC['H_tr'], yp, color='black', label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], HC['GDP_model'][ii]), size=20) 
ax.set_xlabel("Human capital of teachers - Model", fontsize=20)
ax.set_ylabel("GDP - Model", fontsize=20)
ax.grid(True)
plt.tight_layout()    

In [118]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group5:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(TW['twg'], TW['p_tr'], 1)
yp = np.polyval([m, b], TW['twg'])
ax.plot(TW['twg'], yp, color='black', label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper left", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (TW['twg'][ii], TW['p_tr'][ii]), size=20) 
ax.set_xlabel("Teachers wage relative to other occupations - Model", fontsize=20)
ax.set_ylabel("Proportion of teachers - Model", fontsize=20)
ax.grid(True)
plt.tight_layout() 

In [119]:
fig, ax = plt.subplots(figsize=(10,8))
#ax[1].margins(0.05) 
for income, group in group4:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=income)   
(m, b) = np.polyfit(HC['H_tr'], HC['GDP_model'], 1)
yp = np.polyval([m, b], HC['H_tr'])
ax.plot(HC['H_tr'], yp, color='black', label='Regression Line')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(loc="upper right", prop={'size': 12})
for ii, txt in enumerate(names2):
    ax.annotate(txt, (HC['H_tr'][ii], HC['GDP_model'][ii]), size=20) 
ax.set_xlabel("Average human capital of teachers - Model", fontsize=20)
ax.set_ylabel("GDP - Model", fontsize=20)
ax.grid(True)
plt.tight_layout()    